# Análisis de oferta

Solución para hacer análisis de oferta del modelo de abastecimiento estratégico.

In [13]:
import pandas as pd
import numpy as np
from ipywidgets import interact, interactive, fixed, interact_manual
import ipywidgets as widgets
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px

# Paquetes de consulta a traves de Socrata

from sodapy import Socrata
identificador_columnas=pd.read_csv('datos/identificador_columnas.csv')
identificador_columnas=identificador_columnas[['Unificado',valor_columnas[i], 'API_DA_S_II']]
identificador_columnas=identificador_columnas.dropna()
identificador_columnas=identificador_columnas.set_index('Unificado')
identificador_columnas.drop('Fuente',axis=0,inplace=True)
identificador_datos={'SECOP I':'f789-7hwg','SECOP II':'jbjy-vk9h','TVEC':'rgxm-mmea'}
valor_columnas={'SECOP I':valor_columnas[i],'SECOP II':'API_DA_S_II','TVEC':'API_DA_S_TVEC'}
### Definición de cliente para Socrata

cliente=Socrata('www.datos.gov.co',None,timeout=1000000)


In [14]:
identificador_columnas

,API_DA_S_I,API_DA_S_II
Unificado,,
ID Contrato,numero_de_contrato,id_contrato
ID Proceso,uid,proceso_de_compra
ID entidad Plataforma,c_digo_de_la_entidad,codigo_entidad
Entidad,nombre_entidad,nombre_entidad
NIT Entidad,nit_de_la_entidad,nit_entidad
Orden,nivel_entidad,orden
Modalidad,modalidad_de_contratacion,modalidad_de_contratacion
Estado,estado_del_proceso,estado_contrato
Descripción proceso,detalle_del_objeto_a_contratar,descripcion_del_proceso


In [22]:
Queries_list=[]
results=[]
for i in ['SECOP I','SECOP II']:
    query_SECOP="""
    SELECT 
        {2} as entidad,
        {3} as departamento_entidad,
        {4} as orden,
        count({0}) as cantidad,
        sum({1}) as suma_valor_contrato
    Group by 
        {2},{3},{4}
    LIMIT 
        1000000
    """.format(identificador_columnas[valor_columnas[i]]['ID Contrato'],identificador_columnas[valor_columnas[i]]['Valor del contrato'],
    identificador_columnas[valor_columnas[i]]['Entidad'],identificador_columnas[valor_columnas[i]]['Departamento Entidad'],
    identificador_columnas[valor_columnas[i]]['Orden'])
    Queries_list.append(query_SECOP)
    results_detalle = cliente.get(identificador_datos[i], query=query_SECOP)
    results.append(results_detalle)


In [34]:
results_dataframe=pd.concat([pd.DataFrame(results[0]),pd.DataFrame(results[1])])
results_dataframe.reset_index(inplace=True)
results_dataframe.drop('index',axis=1,inplace=True)
results_dataframe['suma_valor_contrato']=results_dataframe['suma_valor_contrato'].astype(float)
results_dataframe['cantidad']=results_dataframe['cantidad'].astype(int)
results_dataframe['orden']=results_dataframe['orden'].str.upper()
results_dataframe['departamento_entidad']=results_dataframe['departamento_entidad'].replace('Distrito Capital de Bogotá','Bogotá DC')\
    .replace('San Andrés Providencia y Santa Catalina','San Andrés, Providencia y Santa Catalina').replace('Norte De Santander','Norte de Santander')

In [40]:
dptos={}
for i in results_dataframe['departamento_entidad'].unique():
    dptos.update({i:[i]})
dptos['Bogotá DC']=['Bogotá DC','Distrito Capital de Bogotá']
dptos['San Andrés, Providencia y Santa Catalina']=['San Andrés, Providencia y Santa Catalina','San Andrés Providencia y Santa Catalina']
dptos['Norte de Santander']=['Norte de Santander','Norte De Santander']
 


In [66]:
def consulta_departamento(departamento):
    results_detalle=[]
    for i in ['SECOP I','SECOP II']:
        queries_list=[]
        results_detalle=[]
        for j in dptos[departamento]:
            results_detalle_dpto=[]
            query_secop_tot=""" Select
                {0} as id_contrato,
                {1} as id_proceso,
                {2} as valor_contrato,
                {3} as entidad,
                {4} as departamento_entidad,
                {5} as orden,
                {6} as modalidad,
                {7} as tipo_contrato,
                {8} as codigo_unspsc,
                {9} as objeto_contrato,
                {10} as descripcion_proceso,
                {11} as nombre_proveedor,
                {12} as tipo_documento_proveedor,
                {13} as numero_documento_proveedor,
                {14} as fecha_inicio,
                {15} as fecha_fin,
                {16} as fecha_firma,
                {17} as url

            where
                {4}='{18}'
            limit
                10000000
            """.format(identificador_columnas[valor_columnas[i]]['ID Contrato'],identificador_columnas[valor_columnas[i]]['ID Proceso'],
            identificador_columnas[valor_columnas[i]]['Valor del contrato'],
            identificador_columnas[valor_columnas[i]]['Entidad'],identificador_columnas[valor_columnas[i]]['Departamento Entidad'],
            identificador_columnas[valor_columnas[i]]['Orden'],identificador_columnas[valor_columnas[i]]['Modalidad'],
            identificador_columnas[valor_columnas[i]]['Tipo de contrato'],identificador_columnas[valor_columnas[i]]['UNSPSC'],
            identificador_columnas[valor_columnas[i]]['Objeto Contrato'],identificador_columnas[valor_columnas[i]]['Descripción proceso'],
            identificador_columnas[valor_columnas[i]]['Nombre Proveedor'],identificador_columnas[valor_columnas[i]]['Tipo de documento proveedor'],
            identificador_columnas[valor_columnas[i]]['Documento Proveedor'],identificador_columnas[valor_columnas[i]]['Inicio de contrato'],
            identificador_columnas[valor_columnas[i]]['Fin de contrato'],identificador_columnas[valor_columnas[i]]['Fecha de Firma'],
            identificador_columnas[valor_columnas[i]]['URL'],j)
            queries_list.append(query_secop_tot)
            results_detalle_dpto.append(pd.DataFrame(cliente.get(identificador_datos[i], query=query_secop_tot)))
        results_detalle.append(pd.concat(results_detalle_dpto))
    results_detalle.append(results_detalle)
    results_detalle=pd.concat(results_detalle)
    results_detalle.reset_index(inplace=True)
    results_detalle.drop('index',axis=1,inplace=True)
    results_detalle['valor_contrato']=results_detalle['valor_contrato'].astype(float)
    results_detalle['departamento_entidad']=results_detalle['departamento_entidad'].replace('Distrito Capital de Bogotá','Bogotá DC')\
    .replace('San Andrés Providencia y Santa Catalina','San Andrés, Providencia y Santa Catalina').replace('Norte De Santander','Norte de Santander')
      
    return results_detalle
        

In [67]:
consulta_departamento('Bogotá DC')